# Module Test Template

## Module & Test Description

Design and testing functions relating to areas and centroids of complex concrete regions. Finally, we'll recreate the ConcreteSection class.

### Imports
##### General Imports

In [1]:
import os, sys, pathlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sc
import shapely as sh

In [2]:
import math

##### Extend PYPATH to current folder:
This allows importing libraries from the same folder; <code>pathlib.Path().resolve()</code> returns the path of the current directory.

In [3]:
sys.path.extend([pathlib.Path().resolve()])

Import specific testing modules:

### Input the regions into a *cross-section* array

In [4]:
# The cross-section will be arrays of [width, height]
region_1 = [140, 14]
region_2 = [14, 360]
region_3 = [24, 24]

# Load up each region into an array
cross_section = np.array([region_1, region_2, region_3])
cross_section

array([[140,  14],
       [ 14, 360],
       [ 24,  24]])

In [5]:
# Slicing array columns into width and height arrays
widths = cross_section[:,0]
heights = cross_section[:,1]
widths, heights

(array([140,  14,  24]), array([ 14, 360,  24]))

In [6]:
# Create function to return the width and height arrays
def get_width_and_height_arrays(cross_section):
    widths = cross_section[:,0]
    heights = cross_section[:,1]
    return widths, heights

In [7]:
widths, heights = get_width_and_height_arrays(cross_section)
widths, heights

(array([140,  14,  24]), array([ 14, 360,  24]))

We need a way to create an array where each index is the sum of previous terms with the current term:

In [8]:
sum_array_thru_index = np.zeros(heights.shape[0])
for i in range(sum_array_thru_index.shape[0]):
    sum_array_thru_index[i] = np.sum(heights[0:i + 1])
sum_array_thru_index

array([ 14., 374., 398.])

Similarly, we need a way to create an area for the sum of previous terms, without the current term value:

In [9]:
sum_array_to_index = np.zeros(heights.shape[0])
for i in range(1, sum_array_to_index.shape[0]):
    sum_array_to_index[i] = np.sum(heights[0:i])
sum_array_to_index

array([  0.,  14., 374.])

$ T_n = \displaystyle\sum_{i=1}^{n}k = 1 + 2 + 3 + 4 + \ldots + n $ are called **triangular numbers** --- because we are doing a triangular sum of the indicies, so to speak, perhaps we can honor the function with this name...    

In [10]:
def get_triangular_sum(a_vector, include_current_index: bool = True):
    sum_array = np.zeros(a_vector.shape[0])
    if include_current_index:
        for i in range(sum_array.shape[0]):
            sum_array[i] = np.sum(a_vector[0:i + 1])
    else:
        for i in range(1, sum_array.shape[0]):
            sum_array[i] = np.sum(a_vector[0:i])     
    return sum_array

In [11]:
lower_bound = get_triangular_sum(heights, include_current_index = False)
lower_bound

array([  0.,  14., 374.])

In [12]:
upper_bound = get_triangular_sum(heights, include_current_index = True)
upper_bound

array([ 14., 374., 398.])

#### Total Height
Finally, we need a way to get the total height from the heights array

In [13]:
def get_total_height(heights):
    return sum(heights)

In [14]:
total_height = get_total_height(heights)
total_height

398

### Get Total Area and Center of Gravity

In [15]:
# Let's get the total area
total_area = 0
for i in range(cross_section.shape[0]):
    total_area += cross_section[i][0] * cross_section[i][1]
total_area

7576

In [16]:
# Convert this to a function
def get_total_area(cross_section: np.array):
    total_area = 0
    for i in range(cross_section.shape[0]):
        total_area += cross_section[i][0] * cross_section[i][1]
    return total_area

In [17]:
# Test the new function
test_total_area = get_total_area(cross_section)
test_total_area

7576

### Distance to Centroid from Edge of Section

In [18]:
def get_centroid_distance(cross_section):
    widths, heights = get_width_and_height_arrays(cross_section)
    total_area = get_total_area(cross_section)
    total_moment = 0
    for i in range(cross_section.shape[0]):
        total_moment += (widths[i] * heights[i] * (get_triangular_sum(heights, False)[i] + heights[i] / 2))
    return total_moment / total_area

In [19]:
centroid = get_centroid_distance(cross_section)
centroid

160.21858500527983

We need a way to get the net area and net centroid if the neutral axis is less than the total height of the section. We can use the same formula above and modify the distance for "a":
- If $ a >= h_i $ then we use $ h_i $
- If $ a < h_i $ then we use h_i = 0
- Else we use $ a - h_i $ as the height for region *i*

In [20]:
cut_distance = 500
lower_bound_heights = get_triangular_sum(heights, False)
net_heights = np.zeros(heights.shape[0])
for i in range(net_heights.shape[0]):
    net_heights[i] = min(max(0, cut_distance - lower_bound_heights[i]), heights[i])
net_heights

array([ 14., 360.,  24.])

In [21]:
# Turn this into a new function
def get_net_heights(heights, cut_distance = math.inf):
    lower_bound_heights = get_triangular_sum(heights, False)
    net_heights = np.zeros(heights.shape[0])
    for i in range(net_heights.shape[0]):
        net_heights[i] = min(max(0, cut_distance - lower_bound_heights[i]), heights[i])
    return net_heights

In [22]:
a = 249.98
test_net_heights = get_net_heights(heights, a)
test_net_heights

array([ 14.  , 235.98,   0.  ])

Revise the get_area() and get_centroid_distance() function to accept an optional cut_distance parameter that will give us the net area and centroids

In [23]:
def get_net_area(cross_section: np.array, cut_distance = math.inf):
    widths, heights = get_width_and_height_arrays(cross_section)
    net_heights = get_net_heights(heights, cut_distance)
    count = net_heights.shape[0]
    net_area = 0
    if count != 0:
        for i in range(net_heights.shape[0]):
            net_area += widths[i] * net_heights[i]
    return net_area

In [24]:
test_net_area = get_net_area(cross_section, cut_distance = a)
test_net_area

5263.719999999999

In [25]:
def get_net_centroid(cross_section: np.array, cut_distance = math.inf):
    widths, heights = get_width_and_height_arrays(cross_section)
    net_heights = get_net_heights(heights, cut_distance)
    net_area = get_net_area(cross_section, cut_distance)    
    net_moment = 0
    if net_area != 0:
        for i in range(net_heights.shape[0]):
            net_moment += (widths[i] * net_heights[i] * (get_triangular_sum(net_heights, False)[i] + net_heights[i] / 2))
    return net_moment / net_area

In [26]:
test_net_centroid = get_net_centroid(cross_section, a)
test_net_centroid

85.44869461141552

<span style="color: dodgerblue;">**NOTE**:</span> These functions all work correctly and have been verified.

<span style="color: tomato;">**TODO**:</span> Implement these functions as properties of the *ConcreteSection* family

# Create ConcreteSection Class
This class should contain the necessary information about the sub-shapes within the concrete section
- Widths
- Heights
- Total and net areas as properties
- Total and net centroids as properties
- Gross moment of inertia

In [27]:
class ConcreteSection():
    def __init__(self, widths_and_heights):
        self._cross_section = np.array(widths_and_heights)
        self._region_count = self.get_region_count(widths_and_heights)
        self.is_valid = self.check_if_valid(self.widths, self.heights)
    
    @property
    def cross_section(self):
        return self._cross_section

    @cross_section.setter
    def cross_section(self, new_widths_and_heights):
        self._cross_section = np.array(new_widths_and_heights)

    @property
    def widths(self):
        return self._cross_section[:,0]

    @property
    def heights(self):
        return self._cross_section[:,1]

    @property
    def gross_area(self):
        if self.is_valid:
            area = 0
            for i in range(self.heights.shape[0]):
                area += self.widths[i] * self.heights[i]
            return area
        else:
            return f"ERROR: cannot calulate area because there are {self.widths.shape[0]} width dimensions, and {self.heights.shape[0]} height dimensions."

    @property
    def gross_centroid(self):
        if self.is_valid:
            total_area = self.gross_area
            total_moment = 0
            for i in range(self.heights.shape[0]):
                total_moment += (self.widths[i] * self.heights[i] * (get_triangular_sum(self.heights, False)[i] + self.heights[i] / 2))
            return total_moment / total_area
        else:
            return f"ERROR: cannot calulate area because there are {self.widths.shape[0]} width dimensions, and {self.heights.shape[0]} height dimensions."
    
    @property
    def total_height(self):
        return sum(self.heights)

    @property
    def max_width(self):
        return max(self.widths)

    @property
    def gross_inertia(self):
        areas = np.zeros(self.heights.shape[0])
        centers = np.zeros(self.heights.shape[0])
        center_offsets = np.zeros(self.heights.shape[0])
        self_inertias = np.zeros(self.heights.shape[0])
        total_centroid = self.gross_centroid
        running_height = 0
        parallel_axis_terms = np.zeros(self.heights.shape[0])
        for i in range(self.heights.shape[0]):
            self_inertias[i] = self.widths[i] / 12 * self.heights[i] ** 3
            areas[i] = self.widths[i] * self.heights[i]
            centers[i] = self.heights[i] / 2 + running_height
            center_offsets[i] = centers[i] - total_centroid
            running_height += self.heights[i]
            parallel_axis_terms[i] = areas[i] * center_offsets[i] ** 2
        return sum(self_inertias) + sum(parallel_axis_terms)

    @property
    def radius_of_gyration(self):
        return math.sqrt(self.gross_inertia / self.gross_area)
    
    def get_region_count(self, widths_and_heights):
        widths_and_heights = np.array(widths_and_heights)
        return widths_and_heights.shape[0]

    def append_region(self, additional_regions):
        new_cross_section = np.append(self._cross_section, additional_regions)
        new_cross_section.resize(int(new_cross_section.size / 2), 2)
        self._cross_section = new_cross_section
        self._region_count = self.get_region_count(self._cross_section)

    def insert_region(self, index, additional_regions):
        if (index == -1) | (index > self._region_count):
            self._cross_section = np.vstack((self._cross_section, np.array(additional_regions)))
        else:
            new_cross_section = np.insert(self._cross_section, index, additional_regions, axis=0)
            self._cross_section = new_cross_section
        self._region_count = self.get_region_count(self._cross_section)

    def delete_region(self, index):
        if index > self._region_count:
            index = self._region_count
        elif index == 0:
            index = 1
        if self._region_count != 1:
            self._cross_section = np.delete(self._cross_section, index - 1, 0)
            self._region_count = self.get_region_count(self._cross_section)
        
    # Cannot make this a property and make net_distance optional
    def area(self, net_distance = math.inf):
        if self.is_valid:
            net_heights = get_net_heights(self.heights, net_distance)
            count = net_heights.shape[0]
            net_area = 0
            if count != 0:
                for i in range(net_heights.shape[0]):
                    net_area += self.widths[i] * net_heights[i]
            return net_area
        else:
            return f"ERROR: cannot calulate area because there are {self.widths.shape[0]} width dimensions, and {self.heights.shape[0]} height dimensions."

    # Cannot make this a property and make net_distance optional
    def centroid(self, net_distance = math.inf):
        if self.is_valid:   
            net_heights = get_net_heights(self.heights, net_distance)
            net_area = self.area(net_distance)
            net_moment = 0
            if net_area != 0:
                for i in range(net_heights.shape[0]):
                    net_moment += (self.widths[i] * net_heights[i] * (get_triangular_sum(net_heights, False)[i] + net_heights[i] / 2))
            return net_moment / net_area
        else:
            return f"ERROR: cannot calulate centroid because there are {self._widths.shape[0]} width dimensions, and {self._heights.shape[0]} height dimensions."

    def get_width_and_height_arrays(self, cross_section):
        cross_section = np.array(cross_section)
        widths = cross_section[:,0]
        heights = cross_section[:,1]
        return widths, heights
        
    def check_if_valid(self, widths, heights):
        return True if widths.shape[0] == heights.shape[0] else False
        
    def get_net_heights(heights, net_distance = math.inf):
        lower_bound_heights = get_triangular_sum(heights, False)
        net_heights = np.zeros(heights.shape[0])
        for i in range(net_heights.shape[0]):
            net_heights[i] = min(max(0, net_distance - lower_bound_heights[i]), heights[i])
        return net_heights
    
    def get_triangular_sum(a_vector, include_current_index: bool = True):
        sum_array = np.zeros(a_vector.shape[0])
        if include_current_index:
            for i in range(sum_array.shape[0]):
                sum_array[i] = np.sum(a_vector[0:i + 1])
        else:
            for i in range(1, sum_array.shape[0]):
                sum_array[i] = np.sum(a_vector[0:i])     
        return sum_array

In [28]:
# The cross-section will be arrays of [width, height]
region_1 = [140, 14]
region_2 = [14, 360]
region_3 = [24, 24]

# Load up each region into an array
cross_section = np.array([region_1, region_2, region_3])
cross_section

array([[140,  14],
       [ 14, 360],
       [ 24,  24]])

In [29]:
sect = ConcreteSection(cross_section)
sect.widths, sect.heights

(array([140,  14,  24]), array([ 14, 360,  24]))

In [30]:
sect.cross_section

array([[140,  14],
       [ 14, 360],
       [ 24,  24]])

In [31]:
new_region = [23, 18]
regions = np.append(sect.cross_section, new_region)
regions.resize(int(regions.size / 2), 2)
regions

array([[140,  14],
       [ 14, 360],
       [ 24,  24],
       [ 23,  18]])

In [32]:
regions = np.append(sect.cross_section, [[10,2],[18,2]])
regions.resize(int(regions.size / 2), 2)
regions

array([[140,  14],
       [ 14, 360],
       [ 24,  24],
       [ 10,   2],
       [ 18,   2]])

In [33]:
sect.append_region([23, 18])
sect.cross_section

array([[140,  14],
       [ 14, 360],
       [ 24,  24],
       [ 23,  18]])

In [34]:
#sect.insert_region(0, [100, 100])
#sect.cross_section

In [35]:
sect.insert_region(1, [100, 100])
sect.cross_section

array([[140,  14],
       [100, 100],
       [ 14, 360],
       [ 24,  24],
       [ 23,  18]])

In [36]:
sect.delete_region(0)
sect.cross_section

array([[100, 100],
       [ 14, 360],
       [ 24,  24],
       [ 23,  18]])

In [37]:
sect.widths, sect.heights

(array([100,  14,  24,  23]), array([100, 360,  24,  18]))

In [38]:
sect.is_valid

True

In [39]:
sect.gross_area, sect.gross_centroid

(16030, 148.9191515907673)

In [40]:
a = 125
sect.area(a), sect.centroid(a)

(10350.0, 52.11352657004831)

In [41]:
sect.area(), sect.centroid()

(16030.0, 148.9191515907673)

In [42]:
sect.max_width, sect.total_height

(100, 502)

In [43]:
new_regions = [[10, 1], [1, 9]]
new_section = ConcreteSection(new_regions)
new_section.gross_inertia, new_section.radius_of_gyration

(180.0043859649123, 3.0779725552358097)

In [44]:
math.sqrt(new_section.gross_inertia / 19)

3.0779725552358097